In [37]:
import mysql.connector as mysql
from mysql.connector import Error
import pandas as pd
import numpy as np

In [38]:
dataset = pd.read_csv("Airplane_Crashes_and_Fatalities_Since_1908.csv")

In [39]:
db_name='airplane_crashes'
try:
    mydb = mysql.connect(host='localhost', user='root', password='Giuliagiulia3') # you can add the auth_plugin here too (ref line 26)
    if mydb.is_connected():
        mycursor = mydb.cursor()
        mycursor.execute('SHOW DATABASES')
        result = mycursor.fetchall()
        print(result)
        for x in result:
            if db_name == x[0]:
                mycursor.execute('DROP DATABASE ' + db_name) # delete old database
                mydb.commit() # make the changes official
                print("The database already exists! The old database has been deleted!)")

        mycursor.execute("CREATE DATABASE "+ db_name)
        print("Database is created")
except Error as e:
    print("Error while connecting to MySQL", e)

[('airplane_crashes',), ('exercise',), ('flights',), ('goshare',), ('information_schema',), ('mysql',), ('performance_schema',), ('sakila',), ('sys',), ('world',)]
The database already exists! The old database has been deleted!)
Database is created


In [40]:
mycursor.execute('USE' + ' ' + db_name)

In [41]:
#create tables
mycursor.execute(
      '''
        CREATE TABLE `Airplane` (
          `serial_number` varchar(100),
          `registration` varchar(100),
          `operator` varchar(100),
          `type` varchar(100),
          PRIMARY KEY (serial_number, registration)
        );
      '''
      )

In [42]:
mycursor.execute(
      '''
        CREATE TABLE Flight (
           flight_number varchar(100),
           aboard int,
           route varchar(100),
           serial_number varchar(100),
           registration varchar(100),
           PRIMARY KEY (flight_number),
           FOREIGN KEY (`serial_number`, `registration`) REFERENCES `Airplane` (`serial_number`, `registration`)
        );
      '''
      )

In [43]:
mycursor.execute(
    '''
        CREATE TABLE `Crash` (
          `index` int,
          `location` varchar(100),
          `time` time NULL ,
          `date` date,
          `summary` varchar(2000),
          `serial number` varchar(100),
          `registration` varchar(100),
          PRIMARY KEY (`index`),
          FOREIGN KEY (`serial number`, `registration`) REFERENCES `Airplane`(`serial_number`, `registration`)
        );
    '''
)

In [44]:
mycursor.execute(
    '''
    CREATE TABLE `Fatalities` (
    `number_of_fatalities` int,
    `ground` int,
    `crash_index` int,
    PRIMARY KEY (`crash_index`),
    FOREIGN KEY (`crash_index`) REFERENCES `Crash`(`index`)
);
    '''
)

In [45]:
selection = dataset.iloc[3196:] #take values from 1980-90 and take all columns except from the first
null_val = selection.isnull().sum()
selection = selection.drop(labels='Flight #', axis=1)

In [46]:
import random

flight_number = []

while len(flight_number) != 2072:
    n = random.randint(1000, 10000)
    if n not in flight_number:
        flight_number.append(n)


selection.insert(6, 'Flight#', flight_number)



serial_num = selection.loc[3196:, 'cn/In']
serial_num_ = list(serial_num)
selection = selection.drop(labels='cn/In', axis=1)

for x in range(len(serial_num_)):
    n = random.randint(100, 8000)
    if serial_num_[x] is np.nan:
        while n in serial_num:
            n = random.randint(100, 8000)
        serial_num_[x] = str(n)

selection.insert(9, 'Serial#', serial_num_)


registration = selection.loc[3196:, 'Registration']
registration_ = list(registration)
selection = selection.drop(labels='Registration', axis=1)

import string
for r in range(len(registration_)):
    reg1 = ''.join(random.choice(string.ascii_uppercase) for x in range(2))
    reg2 = ''.join(random.choice(string.ascii_uppercase) for x in range(3))
    reg = reg1+'-'+reg2
    if reg not in registration_:
        registration_[r] = reg

selection.insert(8, 'Registration', registration_)

import datetime
date = selection.loc[3196:, 'Date']
date_list = list(date)
for i in date_list:
    dates = datetime.datetime.strptime(i, '%m/%d/%Y')
    new = dates.strftime('%Y/%m/%d')
selection = selection.drop(labels='Date', axis=1)
selection.insert(1, 'Date', new)


times = selection['Time'].tolist()
t2 = []
for t in times:
    if type(t)!=str:
            t2.append(0)
    else:
        allowed_characters = ['0','1','2','3','4','5','6','7','8','9',':']
        for i in t:
            if i not in allowed_characters or (i == ':' in (t[0] or t[-1])):
                t = t.replace(i,'')
        if ':' not in t:
            t = t[:len(t)//2]+':'+t[len(t)//2:]
        p = t.partition(':')
        if len(p[0]) > 2:
            t = t[1:]
        if len(p[2]) > 2:
            t = t[:-1]
        try:
            time = datetime.datetime.strptime(t, '%H:%M').time()
            t2.append(time)
        except:
            t2.append(0)
            continue
selection = selection.drop(labels='Time', axis=1)
selection.insert(1, 'Time', t2)

In [47]:
datairplane = selection[['Serial#', 'Registration', 'Operator', 'Type']]

In [48]:
selection.head(2000)

,index,Time,Date,Location,Operator,Route,Flight#,Type,Registration,Serial#,Aboard,Fatalities,Ground,Summary
3196,3196,0,2009/06/08,Haiti,Haïti Air Inter,Port au Prince - Cape Haitien,9001,Britten Norman BN Islander,KM-OJN,474,9.0,9.0,0.0,Struck high ground in poor visibility while en...
3197,3197,01:23:00,2009/06/08,Over the AtlantiOcean,Private charter,Shreveport - Baton Rouge,9801,Cessna 441 Conquest,SK-VBB,6695,2.0,2.0,0.0,The plane was on a 45 minute fight from Shreve...
3198,3198,19:10:00,2009/06/08,"Elburz Mtns., near Laskarak, Markazi, Iran",Iran National Airlines,Babol Sar - Tehran,6993,Boeing B-727-86,WM-LGT,19817/537,128.0,128.0,0.0,The aircraft crashed into a mountain during a ...
3199,3199,0,2009/06/08,"Mt. Cemonyet, Indonesia",Pelita Air Service,NaN,8687,CASA 212 Aviocar 100,QQ-BWZ,45/3N,13.0,13.0,0.0,"Hit high ground at 2,700 ft. in poor weather w..."
3200,3200,06:20:00,2009/06/08,"Near Mandalay, Burma",Military - Union of Burma Air Force,Mandalay - Rangoon,6894,Fairchild FH-227H,VG-NRW,545,44.0,43.0,0.0,Crashed into a building while attempting an em...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5191,5191,16:16:00,2009/06/08,"Off Macae , Rio de Janeiro , Brazil",BHS - Brazilian Helicopter Service Air Taxi,Oil Rig P18 - Macae,7949,Eurocopter AS332L2 Super Puma,SG-RAV,2570,20.0,5.0,0.0,Less than half minute after the taking-off fro...
5192,5192,16:00:00,2009/06/08,"Near Bethani, Nepal",United Nations Mission,NaN,5811,Mi-17,YW-VFX,5588,10.0,10.0,0.0,"The helicopter, carrying United Nations person..."
5193,5193,08:15:00,2009/06/08,"Nbagu, Nigeria",Trade Wings Aviation Ltd.,Lagos - Bebi Air Strip,9864,Beechcraft 1900D,IB-TGH,UE-322,3.0,3.0,0.0,The plane crashed while en route. Wreckage was...
5194,5194,11:00:00,2009/06/08,"Lawa, Suriname",Blue Wing Airlines,Paramaribo - Lawa,1572,Antonov An-28,XZ-BZK,1AJ007-17,19.0,19.0,0.0,While attempting to land the crew aborted the ...


In [49]:
for i,row in datairplane.iterrows():
    sql = "INSERT INTO airplane_crashes.Airplane VALUES (%s,%s,%s,%s)"
    mycursor.execute(sql, tuple([row['Serial#'], row['Registration'], row['Operator'], row['Type']]))
    mydb.commit()

ProgrammingError: 1054 (42S22): Unknown column 'nan' in 'field list'

In [ ]:
#mycursor.execute("SELECT * FROM Airplane")
#result = mycursor.fetchall()
#print(result)

In [ ]:
dataflight = selection[['Flight#', 'Aboard', 'Route', 'Serial#', 'Registration']]

In [ ]:
for i, row in dataflight.iterrows():
    sql = "INSERT INTO airplane_crashes.Flight VALUES (%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple([row['Flight#'], row['Aboard'], row['Route'], row['Serial#'], row['Registration']]))

    mydb.commit()

In [ ]:
datacrash = selection[['index', 'Location', 'Time', 'Date', 'Summary', 'Serial#', 'Registration']]

In [ ]:
for i, row in datacrash.iterrows():
    sql = "INSERT INTO airplane_crashes.Crash VALUES (%s,%s,%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple([row['index'], row['Location'], row['Time'], row['Date'], row['Summary'], row['Serial#'], row['Registration']]))

    mydb.commit()